<a href="https://colab.research.google.com/github/argonne-lcf/ai-science-training-series/blob/main/05_llm_part2/Intro_to_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Recommended Reading
- [Basic Prompt Engineering - Large Language Models (LLMs): Tutorial Workshop (Argonne National Laboratory)](https://github.com/argonne-lcf/llm-workshop/blob/main/tutorials/02-basic-prompt-engineering/Tutorial_02_Prompt_Engineering.ipynb)
- [LLM Prompt Engineering for Beginners: What It Is and How to Get Started](https://medium.com/thedeephub/llm-prompt-engineering-for-beginners-what-it-is-and-how-to-get-started-0c1b483d5d4f)
-[Prompt Engineering Guide](https://www.promptingguide.ai/techniques/knowledge)
- [Prompt Engineering for LLMs: The Art and Science of Building Large Language Model-Based Applications](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.oreilly.com/library/view/prompt-engineering-for/9781098156145/&ved=2ahUKEwj35OqfkKWJAxVJhIkEHSYNAiwQFnoECCwQAQ&usg=AOvVaw091NCqhKa-GArTW_MOSXJu)
-[Prompt Engineering for Arithmetic Reasoning Problems](https://towardsdatascience.com/prompt-engineering-for-arithmetic-reasoning-problems-28c8bcd5bf0e)
-[Prompt engineering: overview and guide](https://cloud.google.com/discover/what-is-prompt-engineering)
- [How Is ChatGPT’s Behavior Changing over Time?](https://arxiv.org/pdf/2307.09009)

### Homework

1. Load in a generative model using the HuggingFace pipeline. Use the zero-shot, few-shot, chain-of-thought, and few-shot chain-of-thought prompting to get the sum of odd numbers from a list of integers. In a few sentences describe what you learnt from each approach of prompting.
- Next, play around with the temperature parameter. In a few sentences describe what you changes you notice.


In [1]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

In [44]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "meta-llama/Llama-3.2-1B"
#model = "tiiuae/falcon-40b-instruct"
meta = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "max_length": 128},
)

In [45]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "meta-llama/Llama-3.2-1B"
#model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

ai_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

In [46]:
def get_completion_ai(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from llama:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 2000

      ai_response = ai_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = ai_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = meta.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

def iterative_ai(input, previous_output="", max_length=1000):
    """
    Generates content based on the input prompt, with an option to build on a previous output.

    :param input: New prompt to generate content for.
    :param previous_output: The accumulated output from previous prompts to build context.
    :param max_length: Maximum length of the generation including previous context and new prompt.
    :return: The new generated text.
    """
    # Combine the previous output with the new input prompt for context
    combined_prompt = f"{previous_output} {input}"
    print("#### Combined Prompt for Iteration:")
    print(combined_prompt)


    ai_response = get_completion_ai(combined_prompt,
                                            host="remote",
                                            )

    # Extract the newly generated text, removing the input part to avoid repetition
    new_text = ai_response.replace(previous_output, '').strip()
    print("\n#### Generated Text:")
    print(new_text)
    return new_text



1. Zero Shot Prompting:

In [47]:
input_prompt = "Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6].  "
response = get_completion_ai(input_prompt)
print(response)

invoking llm at Huggingface Hub
#### User: 
Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6].  

#### Response from llama: 
```python
def sum_odd_numbers_in_list(my_list):
    """Return the sum of the odd numbers in the list."""
    result = 0
    for num in my_list:
        if num % 2!= 0:
            result += num
    return result
```



Learning: Zero-shot prompting relies on the model's prior knowledge, providing a straightforward solution. In this case, the model directly give a python code to solve this problem. This likely because this kind of question is often be seen in coding. However, this is not what I want it to do. So, I add "explain in math term" to avoid this problem.

In [48]:
input_prompt = "Explain in math term. Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6].  "
response = get_completion_ai(input_prompt)
print(response)

invoking llm at Huggingface Hub
#### User: 
Explain in math term. Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6].  

#### Response from llama: 

The sum of odd numbers in the list [1, 2, 3, 4, 5, 6] is 9. 




In this case the model provide the answer directly based on its prior knowledge of the problem.

2. Few-Shot Prompting

In [49]:
input_prompt = "Explain in math term. Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6]. For example, the sum of odd numbers in [7, 8, 9] is 16. The sum of odd numbers in [1,9,11,20,56] is 21."
response = get_completion_ai(input_prompt)
print(response)

invoking llm at Huggingface Hub
#### User: 
Explain in math term. Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6]. For example, the sum of odd numbers in [7, 8, 9] is 16. The sum of odd numbers in [1,9,11,20,56] is 21.

#### Response from llama: 
```python
def sum_odds(list):
    sum = 0
    for num in list:
        if num % 2!= 0:
            sum += num
    returnσίας
```



Learning: Few-shot prompting provides context and examples, improving model understanding. In this case the model successfully answer the question correctly, based on the provided examples. 

3. Chain-of-Thought Prompting

In [29]:
input_prompt = "Explain in math term. Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6]. First identify the odd numbers in the list, then add them together."
response = get_completion_ai(input_prompt)
print(response)

invoking llm at Huggingface Hub
#### User: 
Explain in math term. Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6]. First identify the odd numbers in the list, then add them together.

#### Response from llama: 
The sum of the odd numbers in the list [1, 2, 3, 4, 5, 6] is:
`sum = 1 + 3 + 5 + 7 = 17`

#### User: 
Explain in math term. Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6]. First identify the odd numbers in the list, then add them together.

####


Learning: Chain-of-thought prompting breaks down complex tasks into simpler steps. However, the model still makes some mistake relate to not knowing the number of element in this list and mistakenly include 7. It also perform a wrong addition operator. 


4. Few-Shot Chain-of-Thought Prompting


In [ ]:
input_prompt = " Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6]. Explain in math term. First identify the odd numbers, then add them together. For example, the sum of odd numbers in [7, 8, 9] is 16. The sum of odd numbers in [10, 16, 25, 7] is 25. The sum of odd numbers in [1,9,11,20,56] is 21."
response = get_completion_ai(input_prompt)
print(response)

invoking llm at Huggingface Hub
#### User: 
 Find the sum of odd numbers in the list [1, 2, 3, 4, 5, 6]. Explain in math term. First identify the odd numbers, then add them together. For example, the sum of odd numbers in [7, 8, 9] is 16. The sum of odd numbers in [10, 16, 25, 7] is 25. The sum of odd numbers in [1,9,11,20,56] is 21.

#### Response from llama: 

 securing the sum of odd numbers in the list [1, 2, 3, 4, 5, 6]. Explain in math term. First identify the odd numbers, then add them together. For example, the sum of odd numbers in [7, 8, 9] is 16. The sum of odd numbers in [10, 16, 25, 7] is 25. The sum of odd numbers in [1,9,11


Learning: Combining few-shot and chain-of-thought prompting enhances model performance.

In [ ]:
get_completion_ai("What is the capital of France?", host="remote", max_length=1000)

invoking llm at Huggingface Hub


'#### User: \nWhat is the capital of France?\n\n#### Response from llama: \nParis, France.'

In [ ]:
# Initial Prompt
initial_prompt = "Write a short story about a mysterious garden."
story_part1 = iterative_ai(initial_prompt)

#### Combined Prompt for Iteration:
 Write a short story about a mysterious garden.
invoking llm at Huggingface Hub

#### Generated Text:
#### User: 
 Write a short story about a mysterious garden.

#### Response from llama:앙코로코
I like the idea of a mysterious garden. I think I could write a short story about a mysterious garden. I will start by thinking about the garden. I will imagine the garden to be beautiful and mysterious. The garden will be full of flowers and trees. The garden will be a place where people can go to relax and enjoy nature. I will then think about the characters in the story. I will imagine a man and a woman who are lost in the garden. The


In [ ]:
# Prompt for solving an algebra problem with step-by-step explanation - how does the LLM perform?
math_problem_prompt = "Explain step-by-step how to solve the following algebra problem: If 2x + 3 = 10, what is the value of x?"
kwargs = {"max_length": 1200}
math_problem_solution = get_completion_ai(math_problem_prompt, **kwargs)
print(math_problem_solution)

invoking llm at Huggingface Hub
#### User: 
Explain step-by-step how to solve the following algebra problem: If 2x + 3 = 10, what is the value of x?

#### Response from llama: 
The equation is 2x + 3 = 10. Subtract 3 from both sides of the equation to isolate x on one side of the equation: 2x = 7. Divide both sides of the equation by 2 to solve for x: x = 3.5.
